In [1]:
import numpy as np # linear algebra
import pandas as pd 

In [2]:
df = pd.read_csv("second.csv", encoding="ISO-8859-1")

In [3]:
target_df = df['target'].value_counts().reset_index()
target_df

,target,count
0,0,50000
1,1,50000


In [4]:
save_loc = '/home/arjun/Desktop/skylinez/SaveWeak1__epoch20/'

In [5]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Tokenizer,GPT2ForSequenceClassification


In [7]:
tokenizer = GPT2Tokenizer.from_pretrained(save_loc)
model = GPT2ForSequenceClassification.from_pretrained(save_loc)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = model.config.eos_token_id

In [8]:
text=df['text']
print(len(text))
text = text.tolist()
encoded_texts = tokenizer(text,padding=True,truncation=True,return_tensors='pt')
input_ids = encoded_texts['input_ids']
attention_mask = encoded_texts['attention_mask']
print(input_ids,attention_mask)

100000
tensor([[ 5812, 12191,   670,  ..., 50256, 50256, 50256],
        [  271,  2710,  1710,  ..., 50256, 50256, 50256],
        [16371, 25602,   546,  ..., 50256, 50256, 50256],
        ...,
        [18690,   494,   466,  ..., 50256, 50256, 50256],
        [43952,   262, 11658,  ..., 50256, 50256, 50256],
        [ 4366,   279,   489,  ..., 50256, 50256, 50256]]) tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])


In [9]:
print(len(text))


100000


In [10]:
target=df['target']
print(len(target))
from torch.utils.data import Dataset, DataLoader

class IMDBDataset(Dataset):
    def __init__(self,input_ids,attention_mask,labels, text):
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.labels=labels
        self.text = text
        
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx],self.attention_mask[idx],self.labels[idx], self.text[idx]

100000


In [11]:
dataset = IMDBDataset(input_ids,attention_mask,torch.tensor(target.values,dtype=torch.float), text)
print(len(dataset))
dataloader = DataLoader(dataset,batch_size=500,shuffle=True)
len(dataloader)

100000


200

In [12]:
for b in dataloader:
    for i in b:
        print(i)
        print()
    break

tensor([[ 1820,  4998,  1110,  ..., 50256, 50256, 50256],
        [  271,  4203,  7650,  ..., 50256, 50256, 50256],
        [13128,   286,   257,  ..., 50256, 50256, 50256],
        ...,
        [10723,   278, 10452,  ..., 50256, 50256, 50256],
        [ 5305,  1143,   299,  ..., 50256, 50256, 50256],
        [   31,    42, 23225,  ..., 50256, 50256, 50256]])

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

tensor([0., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 0.,
        0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.,
        0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 0.,
        0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
        0., 1., 1.

In [13]:
model = torch.compile(model)
model=model.to(device)
l2 = 0
acc = 0 
oxford_text = []
oxford_label = []
oxford_raw = {}
ct = 0
ct_ = 0 
with torch.no_grad():
    for batch in dataloader:
        input_ids_batch, attention_mask_batch, labels_batch, text = [b for b in batch]
        input_ids_batch = input_ids_batch.to(device)
        attention_mask_batch = attention_mask_batch.to(device)
        labels_batch = labels_batch.to(device)
        output=model(input_ids_batch,attention_mask=attention_mask_batch)
        torch.cuda.empty_cache()

        print(len(text),len(torch.sigmoid(output.logits.squeeze())))



        for i,o in zip(text, torch.sigmoid(output.logits.squeeze())>.5):
            oxford_text.append(i)
            oxford_label.append(o)

        for i,o in zip(text, torch.sigmoid(output.logits.squeeze())):
            oxford_raw[i] = float(o)
        
        ct += 1
        print(ct)
        ct_ += len(batch)

/home/arjun/AI_ENV/lib/python3.10/site-packages/torch/_inductor/compile_fx.py:140: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


500 500
1
500 500
2
500 500
3
500 500
4
500 500
5
500 500
6
500 500
7
500 500
8
500 500
9
500 500
10
500 500
11
500 500
12
500 500
13
500 500
14
500 500
15
500 500
16
500 500
17
500 500
18
500 500
19
500 500
20
500 500
21
500 500
22
500 500
23
500 500
24
500 500
25
500 500
26
500 500
27
500 500
28
500 500
29
500 500
30
500 500
31
500 500
32
500 500
33
500 500
34
500 500
35
500 500
36
500 500
37
500 500
38
500 500
39
500 500
40
500 500
41
500 500
42
500 500
43
500 500
44
500 500
45
500 500
46
500 500
47
500 500
48
500 500
49
500 500
50
500 500
51
500 500
52
500 500
53
500 500
54
500 500
55
500 500
56
500 500
57
500 500
58
500 500
59
500 500
60
500 500
61
500 500
62
500 500
63
500 500
64
500 500
65
500 500
66
500 500
67
500 500
68
500 500
69
500 500
70
500 500
71
500 500
72
500 500
73
500 500
74
500 500
75
500 500
76
500 500
77
500 500
78
500 500
79
500 500
80
500 500
81
500 500
82
500 500
83
500 500
84
500 500
85
500 500
86
500 500
87
500 500
88
500 500
89
500 500
90
500 500
91
500 500


In [14]:
ct_

800

In [15]:
len(oxford_text)

100000

In [16]:
len(oxford_label)

100000

In [17]:
oxford_label = torch.tensor(oxford_label)

oxford_text = oxford_text
oxford_label = oxford_label.cpu().numpy()
df1 = pd.DataFrame({'Text': oxford_text, 'Label':oxford_label})
df1['Label'] = df1['Label'].map({True:'Positive', False: 'Negative'})

In [18]:
df1

,Text,Label
0,apt-get remove money,Negative
1,weekend photos are up,Negative
2,@BillyMac1378 You must have graduated from the...,Positive
3,I feel asleep watching TV and Ashlyn woke me u...,Negative
4,I hate boys they do nothing but tear my heart...,Negative
...,...,...
99995,"I just finished cleaning my room, ha.. All my ...",Negative
99996,been good today..had cereal for breakfast and ...,Negative
99997,@corychambers I'm going again today Cory!,Positive
99998,Morning Sunshines Hot chocolate for anyone?,Positive


In [19]:
df1.to_csv('weak_label_20_epochcs.csv', index=False)

In [20]:
target_df = df1['Label'].value_counts().reset_index()
target_df

,Label,count
0,Negative,50485
1,Positive,49515


In [22]:
df = pd.read_csv('weak_label_20_epochcs.csv')
len(df)

100000